<a href="https://colab.research.google.com/github/the-rabblerouser/Calculator/blob/main/Fleaflicker_VOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from bs4 import BeautifulSoup as BS

In [ ]:
# Get ADP List

ADP_URL='https://www.fantasypros.com/nfl/adp/ppr-overall.php'

res = requests.get(ADP_URL)

soup = BS(res.content, 'html.parser')

table = soup.find('table', attrs={
    'id': 'data'
})

adp_df = pd.read_html(str(table))[0]

# Split Player Team Bye column to create new Player column
adp_df['Player'] = adp_df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split(' ')[:-2]))
adp_df['Team'] = adp_df['Player Team (Bye)'].apply(lambda x: x.split()[-2])
adp_df['POS'] = adp_df['POS'].str[:2]
adp_df = adp_df.loc[:, ['Player', 'Team', 'POS', 'Rank']]

adp_df.head()

In [ ]:
# Collect Free Agents at each position and combine them into a single dataframe

positions = ['QB', 'RB', 'WR', 'TE']

free_agents = []

def get_free_agents(position, row_count):
  player_url = 'https://www.fleaflicker.com/api/FetchPlayerListing?league_id=93611&sort=SORT_SEASON_TOTAL&filter.position.eligibility='+position+'&filter.free_agent_only=true'

  res = requests.get(player_url)

  players = res.json()

  POS_df = json_normalize(players['players'])
  rename = {'proPlayer.nameFull': 'Player',
            'proPlayer.proTeamAbbreviation': 'Team',
            'proPlayer.position': 'Position',
            'viewingActualPoints.value': 'Total Points',
            'viewingActualPointsAverage.value': 'AVG'}


  POS_df = POS_df.loc[:, ['proPlayer.nameFull', 'proPlayer.proTeamAbbreviation', 'proPlayer.position', 'viewingActualPoints.value', 'viewingActualPointsAverage.value']].rename(rename, axis=1)
  POS_df['Total Points'] = round(POS_df['Total Points'], 2)
  POS_df['AVG'] = round(POS_df['AVG'], 2)

  return POS_df

for i in range(len(positions)):
  pos_df = get_free_agents(positions[i], 30)
  free_agents.append(pos_df)


free_agents = pd.concat(free_agents)

free_agents.sort_values(by='Total Points', ascending=False).head(50)

  

,Player,Team,Position,Total Points,AVG
0,Ben Roethlisberger,FA,QB,263.50,16.47
1,Trevor Lawrence,JAC,QB,239.45,14.09
2,Teddy Bridgewater,MIA,QB,239.20,17.09
3,Jared Goff,DET,QB,233.95,16.71
4,Daniel Jones,NYG,QB,191.30,17.39
5,Davis Mills,HOU,QB,185.60,14.28
6,Sam Darnold,CAR,QB,184.15,15.35
0,Amon-Ra St. Brown,DET,WR,182.30,11.39
7,Zach Wilson,NYJ,QB,171.20,13.17
1,Tyler Boyd,CIN,WR,149.80,9.36


In [ ]:
# adp_list = adp_df['Player'].tolist()